 Memory Management - Part 1 - Sessions¶
Welcome to Day 3 of the Kaggle 5-day Agents course!

In this notebook, you'll learn:

✅ What sessions are and how to use them in your agent
✅ How to build stateful agents with sessions and events
✅ How to persist sessions in a database
✅ Context management practices such as context compaction
✅ Best practices for sharing session Sta

Section 2: Session Management¶
2.1 The Problem
At their core, Large Language Models are inherently stateless. Their awareness is confined to the information you provide in a single API call. This means an agent without proper context management will react to the current prompt without considering any previous history.

❓ Why does this matter? Imagine trying to have a meaningful conversation with someone who forgets everything you've said after each sentence. That's the challenge we face with raw LLMs!

In ADK, we use Sessions for short term memory management and Memory for long term memory. In the next notebook, you'll focus on Memory.

2.2 What is a Session?
📦 Session
A session is a container for conversations. It encapsulates the conversation history in a chronological manner and also records all tool interactions and responses for a single, continuous conversation. A session is tied to a user and agent; it is not shared with other users. Similarly, a session history for an Agent is not shared with other Agents.

In ADK, a Session is comprised of two key components Events and State:

📝 Session.Events:

While a session is a container for conversations, Events are the building blocks of a conversation.

Example of Events:

User Input: A message from the user (text, audio, image, etc.)
Agent Response: The agent's reply to the user
Tool Call: The agent's decision to use an external tool or API
Tool Output: The data returned from a tool call, which the agent uses to continue its reasoning
{} Session.State:

session.state is the Agent's scratchpad, where it stores and updates dynamic details needed during the conversation. Think of it as a global {key, value} pair storage which is available to all subagents and tools.

In [11]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, AgentTool
from google.genai import types
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory




In [12]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")



 How to manage sessions?
An agentic application can have multiple users and each user may have multiple sessions with the application. To manage these sessions and events, ADK offers a Session Manager and Runner.

SessionService: The storage layer

Manages creation, storage, and retrieval of session data
Different implementations for different needs (memory, database, cloud)
Runner: The orchestration layer

Manages the flow of information between user and agent
Automatically maintains conversation history
Handles the Context Engineering behind the scenes
Think of it like this:

Session = A notebook 📓
Events = Individual entries in a single page 📝
SessionService = The filing cabinet storing notebooks 🗄️
Runner = The assistant managing the conversation 🤖

In [13]:
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [14]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [15]:
APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session

MODEL_NAME = "gemini-2.5-flash-lite"


# Step 1: Create the LLM Agent
root_agent = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot",  # Description of the agent's purpose
)

# Step 2: Set up Session Management
# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()

# Step 3: Create the Runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)


In [16]:
await run_session(
    runner,
    [
        "Hi, I am Sam! What is the capital of United States?",
        "Hello! What is my name?",  # This time, the agent should remember!
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Hi, I am Sam! What is the capital of United States?
gemini-2.5-flash-lite >  Hi Sam! The capital of the United States is Washington, D.C.

User > Hello! What is my name?
gemini-2.5-flash-lite >  Your name is Sam!


Section 3: Persistent Sessions with DatabaseSessionService¶
While InMemorySessionService is great for prototyping, real-world applications need conversations to survive restarts, crashes, and deployments. Let's level up to persistent storage!

3.1 Choosing the Right SessionService
ADK provides different SessionService implementations for different needs:

Service	Use Case	Persistence	Best For
InMemorySessionService	Development & Testing	❌ Lost on restart	Quick prototypes
DatabaseSessionService	Self-managed apps	✅ Survives restarts	Small to medium apps
Agent Engine Sessions	Production on GCP	✅ Fully managed	Enterprise scale
3.2 Implementing Persistent Sessions
Let's upgrade to DatabaseSessionService using SQLite. This gives us persistence without needing a separate database server for this demo.

Let's create a chatbot_agent capable of having a conversation with the user.



In [17]:
#  Step 1: Create the same agent (notice we use LlmAgent this time)
chatbot_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot with persistent memory",
)

# Step 2: Switch to DatabaseSessionService
# SQLite database will be created automatically
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Step 3: Create a new runner with persistent storage
runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)


est Run 1: Verifying Persistence¶
In this first test run, we'll start a new conversation with the session ID test-db-session-01. We will first introduce our name as 'Sam' and then ask a question. In the second turn, we will ask the agent for our name.

Since we are using DatabaseSessionService, the agent should remember the name.

After the conversation, we'll inspect the my_agent_data.db SQLite database directly to see how the conversation events (the user queries and model responses) are stored.

In [18]:
await run_session(
    runner,
    ["Hi, I am Sam! What is the capital of the United States?", "Hello! What is my name?"],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > Hi, I am Sam! What is the capital of the United States?
gemini-2.5-flash-lite >  Hi Sam! The capital of the United States is Washington, D.C.

User > Hello! What is my name?
gemini-2.5-flash-lite >  Your name is Sam!


In [19]:
await run_session(
    runner, ["Hello! What is my name?"], "test-db-session-02"
)  # Note, we are using new session name


 ### Session: test-db-session-02

User > Hello! What is my name?
gemini-2.5-flash-lite >  I do not have access to personal information about you, including your name. I am a text chatbot and do not have the ability to remember or retrieve personal details.


How are the events stored in the Database?¶
Since we are using a sqlite DB to store information, let's have a quick peek to see how information is stored.

In [56]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "SELECT app_name, session_id, author, content FROM events"
        )
        print([col[0] for col in result.description])
        for row in result.fetchall():
            print(row)

check_data_in_db()  # connection closes automatically here


['app_name', 'session_id', 'author', 'content']
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hi, I am Sam! What is the capital of the United States?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Hi Sam! The capital of the United States is Washington, D.C."}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Your name is Sam!"}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hi, I am Sam! What is the capital of the United States?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Hi Sam! The capital of the United States is Washington, D.C."}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')
('default', 'test-db

Section 4: Context Compaction¶
As you can see, all the events are stored in full in the session Database, and this quickly adds up. For a long, complex task, this list of events can become very large, leading to slower performance and higher costs.

But what if we could automatically summarize the past? Let's use ADK's Context Compaction feature to see how to automatically reduce the context that's stored in the Session.

4.1 Create an App for the agent
To enable this feature, let's use the same chatbot_agent we created in Section 3.2.

The first step is to create an object called App. We'll give it a name and pass in our chatbot_agent.

We'll also create a new config to do the Context Compaction. This EventsCompactionConfig defines two key variables:

compaction_interval: Asks the Runner to compact the history after every n conversations
overlap_size: Defines the number of previous conversations to retain for overlap
We'll then provide this app to the Runner.

In [57]:
# Re-define our app with Events Compaction enabled
research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    # This is the new part!
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1,  # Keep 1 previous turn for context
    ),
)

db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Create a new runner for our upgraded app
research_runner_compacting = Runner(
    app=research_app_compacting, session_service=session_service
)


C:\Users\Admin\AppData\Local\Temp\ipykernel_7012\441359075.py:6: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


Running the Demo¶
Now, let's have a conversation that is long enough to trigger the compaction. When you run the cell below, the output will look like a normal conversation. However, because we configured our App, a compaction process will run silently in the background after the 3rd invocation.

In [22]:
await run_session(
    research_runner_compacting,
    "What is the latest news about AI in healthcare?",
    "compaction_demo",
)

# Turn 2
await run_session(
    research_runner_compacting,
    "Are there any new developments in drug discovery?",
    "compaction_demo",
)

# Turn 3 - Compaction should trigger after this turn!
await run_session(
    research_runner_compacting,
    "Tell me more about the second development you found.",
    "compaction_demo",
)

# Turn 4
await run_session(
    research_runner_compacting,
    "Who are the main companies involved in that?",
    "compaction_demo",
)


 ### Session: compaction_demo

User > What is the latest news about AI in healthcare?
gemini-2.5-flash-lite >  Here's a summary of some of the latest news and trends in AI in healthcare, as of late 2023 and early 2024:

**Key Areas of Advancement and Investment:**

*   **Drug Discovery and Development:** This remains a major focus. Companies are using AI to:
    *   **Identify novel drug targets:** Analyzing vast biological datasets to pinpoint proteins or genes involved in diseases.
    *   **Design new molecules:** Predicting which chemical structures are most likely to be effective and safe.
    *   **Optimize clinical trials:** Identifying suitable patient cohorts, predicting trial outcomes, and streamlining trial management.
    *   **Examples:** Several AI-powered drug discovery startups have announced significant funding rounds and partnerships with established pharmaceutical giants.
*   **Diagnostic Imaging and Analysis:** AI continues to revolutionize how medical images are i

4.3 Verifying Compaction in the Session History¶
The conversation above looks normal, but the history has been changed behind the scenes. How can we prove it?

We can inspect the events list from our session. The compaction process doesn't delete old events; it replaces them with a single, new Event that contains the summary. Let's find it.

In [23]:
final_session = await session_service.get_session(
    app_name=research_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="compaction_demo",
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("\n✅ SUCCESS! Found the Compaction Event:")
        print(f"  Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n❌ No compaction event found. Try increasing the number of turns in the demo."
    )

--- Searching for Compaction Summary Event ---

✅ SUCCESS! Found the Compaction Event:
  Author: user

 Compacted information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='788a01e4-a99a-4ec0-9247-9454d86a3424' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction={'start_timestamp': 1763402969.055482, 'end_timestamp': 1763402979.540613, 'compacted_content': {'parts': [{'function_call': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_response': None, 'inline_data'

4.4 What you've accomplished: Automatic Context Management
You just found the proof! The presence of that special summary Event in your session's history is the tangible result of the compaction process.

Let's recap what you just witnessed:

Silent Operation: You ran a standard conversation, and from the outside, nothing seemed different.
Background Compaction: Because you configured the App with EventsCompactionConfig, the ADK Runner automatically monitored the conversation length. Once the threshold was met, it triggered the summarization process in the background.
Verified Result: By inspecting the session's events, you found the summary that the LLM generated. This summary now replaces the older, more verbose turns in the agent's active context.
For all future turns in this conversation, the agent will be given this concise summary instead of the full history. This saves costs, improves performance, and helps the agent stay focused on what's most important.

4.5 More Context Engineering options in ADK
👉 Custom Compaction
In this example, we used ADK's default summarizer. For more advanced use cases, you can provide your own by defining a custom SlidingWindowCompactor and passing it to the config. This allows you to control the summarization prompt or even use a different, specialized LLM for the task. You can read more about it in the official documentation.

👉 Context Caching
ADK also provides Context Caching to help reduce the token size of the static instructions that are fed to the LLM by caching the request data. Read more about it here.

The Problem
While we can do Context Compaction and use a database to resume a session, we face new challenges now. In some cases, we have key information or preferences that we want to share across other sessions.

In these scenarios, instead of sharing the entire session history, transferring information from a few key variables can improve the session experience. Let's see how to do it!

🤝 Section 5: Working with Session State¶
5.1 Creating custom tools for Session state management
Let's explore how to manually manage session state through custom tools. In this example, we'll identify a transferable characteristic, like a user's name and their country, and create tools to capture and save it.

Why This Example?

The username is a perfect example of information that:

Is introduced once but referenced multiple times
Should persist throughout a conversation
Represents a user-specific characteristic that enhances personalization
Here, for demo purposes, we'll create two tools that can store and retrieve user name and country from the Session State. Note that all tools have access to the ToolContext object. You don't have to create separate tools for each piece of information you want to share.

In [24]:
# Define scope levels for state keys (following best practices)
USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")


# This demonstrates how tools can write to session state using tool_context.
# The 'user:' prefix indicates this is user-specific data.
def save_userinfo(
    tool_context: ToolContext, user_name: str, country: str
) -> Dict[str, Any]:
    """
    Tool to record and save user name and country in session state.

    Args:
        user_name: The username to store in session state
        country: The name of the user's country
    """
    # Write to session state using the 'user:' prefix for user data
    tool_context.state["user:name"] = user_name
    tool_context.state["user:country"] = country

    return {"status": "success"}


# This demonstrates how tools can read from session state.
def retrieve_userinfo(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Tool to retrieve user name and country from session state.
    """
    # Read from session state
    user_name = tool_context.state.get("user:name", "Username not found")
    country = tool_context.state.get("user:country", "Country not found")

    return {"status": "success", "user_name": user_name, "country": country}



Key Concepts:

Tools can access tool_context.state to read/write session state
Use descriptive key prefixes (user:, app:, temp:) for organization
State persists across conversation turns within the same session
5.2 Creating an Agent with Session State Tools
Now let's create a new agent that has access to our session state management tools:

In [25]:
#  Configuration
APP_NAME = "default"
USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"

# Create an agent with session state tools
root_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="""A text chatbot.
    Tools for managing user context:
    * To record username and country when provided use `save_userinfo` tool. 
    * To fetch username and country when required use `retrieve_userinfo` tool.
    """,
    tools=[save_userinfo, retrieve_userinfo],  # Provide the tools to the agent
)

# Set up session service and runner
session_service = InMemorySessionService()
runner = Runner(agent=root_agent, session_service=session_service, app_name="default")


5.3 Testing Session State in Action
Let's test how the agent uses session state to remember information across conversation turns:

In [26]:
# Test conversation demonstrating session state
await run_session(
    runner,
    [
        "Hi there, how are you doing today? What is my name?",  # Agent shouldn't know the name yet
        "My name is Sam. I'm from Poland.",  # Provide name - agent should save it
        "What is my name? Which country am I from?",  # Agent should recall from session state
    ],
    "state-demo-session",
)


 ### Session: state-demo-session

User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  I'm doing great, thank you for asking! I'd love to tell you your name, but I don't have it at the moment. Could you let me know what it is?


User > My name is Sam. I'm from Poland.


gemini-2.5-flash-lite >  It is a pleasure to meet you, Sam! I've saved your information. How can I help you today?


User > What is my name? Which country am I from?


gemini-2.5-flash-lite >  Your name is Sam and you are from Poland.


5.4 Inspecting Session State
Let's directly inspect the session state to see what's stored:

In [27]:
# Retrieve the session and inspect its state
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="state-demo-session"
)

print("Session State Contents:")
print(session.state)

Session State Contents:
{'user:name': 'Sam', 'user:country': 'Poland'}


5.5 Session State Isolation
As we've already seen, an important characteristic of session state is that it's isolated per session. Let's demonstrate this by starting a new session:

In [28]:
# Start a completely new session - the agent won't know our name
await run_session(
    runner,
    ["Hi there, how are you doing today? What is my name?"],
    "new-isolated-session",
)

# Expected: The agent won't know the name because this is a different session


 ### Session: new-isolated-session

User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  I'm doing great, thank you for asking! I don't know your name yet. Would you like to tell me what it is? Also, where are you from?


5.6 Cross-Session State Sharing
While sessions are isolated by default, you might notice something interesting. Let's check the state of our new session (new-isolated-session):

In [29]:
# Check the state of the new session
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="new-isolated-session"
)

print("New Session State:")
print(session.state)

New Session State:
{'user:name': 'Sam', 'user:country': 'Poland'}



🧹 Cleanup¶

In [59]:
# # Clean up any existing database to start fresh (if Notebook is restarted)
# import os

# if os.path.exists("my_agent_data.db"):
#     os.remove("my_agent_data.db")
# print("✅ Cleaned up old database files")



import sqlite3
import os

db_path = "my_agent_data.db"

# Try to release SQLite lock
try:
    conn = sqlite3.connect(db_path)
    conn.close()
except:
    pass

# Now delete safely
if os.path.exists(db_path):
    try:
        os.remove(db_path)
        print("✅ Cleaned up old database file:", db_path)
    except Exception as e:
        print("❌ Could not delete database file:", e)
else:
    print("ℹ️ No database file found.")


❌ Could not delete database file: [WinError 32] The process cannot access the file because it is being used by another process: 'my_agent_data.db'


🤓 Section 2: Memory Workflow¶
From the Introduction section, you now know why we need Memory. In order to integrate Memory into your Agents, there are three high-level steps.

Three-step integration process:

Initialize → Create a MemoryService and provide it to your agent via the Runner
Ingest → Transfer session data to memory using add_session_to_memory()
Retrieve → Search stored memories using search_memory()
Let's explore each step in the following sections.

🧠 Section 3: Initialize MemoryService¶
3.1 Initialize Memory
ADK provides multiple MemoryService implementations through the BaseMemoryService interface:

InMemoryMemoryService - Built-in service for prototyping and testing (keyword matching, no persistence)
VertexAiMemoryBankService - Managed cloud service with LLM-powered consolidation and semantic search
Custom implementations - You can build your own using databases, though managed services are recommended
For this notebook, we'll use InMemoryMemoryService to learn the core mechanics. The same methods work identically with production-ready services like Vertex AI Memory Bank.

In [32]:
memory_service = (
    InMemoryMemoryService()
)  # ADK's built-in Memory Service for development and testing

3.2 Add Memory to Agent
Next, create a simple agent to answer user queries.

In [33]:
APP_NAME = "MemoryDemoApp"
USER_ID = "demo_user"

# Create agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words.",
)

Create Runner
Now provide both Session and Memory services to the Runner.

Key configuration:

The Runner requires both services to enable memory functionality:

session_service → Manages conversation threads and events
memory_service → Provides long-term knowledge storage
Both services work together: Sessions capture conversations, Memory stores knowledge for retrieval across sessions.

In [34]:
# Create Session Service
session_service = InMemorySessionService()  # Handles conversations

# Create runner with BOTH services
runner = Runner(
    agent=user_agent,
    app_name="MemoryDemoApp",
    session_service=session_service,
    memory_service=memory_service,  # Memory service is now available!
)

Important¶
💡 Configuration vs. Usage: Adding memory_service to the Runner makes memory available to your agent, but doesn't automatically use it. You must explicitly:

Ingest data using add_session_to_memory()
Enable retrieval by giving your agent memory tools (load_memory or preload_memory)
Let's learn these steps next!

3.3 MemoryService Implementation Options
This notebook: InMemoryMemoryService

Stores raw conversation events without consolidation
Keyword-based search (simple word matching)
In-memory storage (resets on restart)
Ideal for learning and local development
Production: VertexAiMemoryBankService (You'll learn this on Day 5)

LLM-powered extraction of key facts
Semantic search (meaning-based retrieval)
Persistent cloud storage
Integrates external knowledge sources
💡 API Consistency: Both implementations use identical methods (add_session_to_memory(), search_memory()). The workflow you learn here applies to all memory services!



Section 4: Ingest Session Data into Memory¶
Why should you transfer Session data to Memory?

Now that memory is initialized, you need to populate it with knowledge. When you initialize a MemoryService, it starts completely empty. All your conversations are stored in Sessions, which contain raw events including every message, tool call, and metadata. To make this information available for long-term recall, you explicitly transfer it to memory using add_session_to_memory().

Here's where managed memory services like Vertex AI Memory Bank shine. During transfer, they perform intelligent consolidation - extracting key facts while discarding conversational noise. The InMemoryMemoryService we're using stores everything without consolidation, which is sufficient for learning the mechanics.

Before we can transfer anything, we need data. Let's have a conversation with our agent to populate the session. This conversation will be stored in the SessionService just like you learned in the previous notebook.

In [35]:
await run_session(
    runner,
    "My favorite color is blue-green. Can you write a Haiku about it?",
    "conversation-01",  # Session ID
)


 ### Session: conversation-01

User > My favorite color is blue-green. Can you write a Haiku about it?
gemini-2.5-flash-lite >  A color so calm,
Like the ocean, cool and deep,
Nature's gentle hue.


In [36]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)

# Let's see what's in the session
print("📝 Session contains:")
for event in session.events:
    text = (
        event.content.parts[0].text[:60]
        if event.content and event.content.parts
        else "(empty)"
    )
    print(f"  {event.content.role}: {text}...")

📝 Session contains:
  user: My favorite color is blue-green. Can you write a Haiku about...
  model: A color so calm,
Like the ocean, cool and deep,
Nature's gen...


Perfect! The session contains our conversation. Now we're ready to transfer it to memory. Call add_session_to_memory() and pass the session object. This ingests the conversation into the memory store, making it available for future searches.

In [37]:
await memory_service.add_session_to_memory(session)

 Section 5: Enable Memory Retrieval in Your Agent¶
You've successfully transferred session data to memory, but there's one crucial step remaining. Agents can't directly access the MemoryService - they need tools to search it.

This is by design: it gives you control over when and how memory is retrieved.

5.1 Memory Retrieval in ADK
ADK provides two built-in tools for memory retrieval:

load_memory (Reactive)

Agent decides when to search memory
Only retrieves when the agent thinks it's needed
More efficient (saves tokens)
Risk: Agent might forget to search
preload_memory (Proactive)

Automatically searches before every turn
Memory always available to the agent
Guaranteed context, but less efficient
Searches even when not needed
Think of it like studying for an exam: load_memory is looking things up only when you need them, while preload_memory is reading all your notes before answering each question.

5.2 Add Load Memory Tool to Agent
Let's start by implementing the reactive pattern. We'll recreate the agent from Section 3, this time adding the load_memory tool to its toolkit. Since this is a built-in ADK tool, you simply include it in the tools array without any custom implementation.

In [38]:
# Create agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words. Use load_memory tool if you need to recall past conversations.",
    tools=[
        load_memory
    ],  # Agent now has access to Memory and can search it whenever it decides to!
)


5.3 Update the Runner and Test
Let's now update the Runner to use our new user_agent that has the load_memory tool. And we'll ask the Agent about the favorite color which we had stored previously in another session.

👉 Since sessions don't share conversation history, the only way the agent can answer correctly is by using the load_memory tool to retrieve the information from long-term memory that we manually stored.

In [39]:
# Create a new runner with the updated agent
runner = Runner(
    agent=user_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

await run_session(runner, "What is my favorite color?", "color-test")


 ### Session: color-test

User > What is my favorite color?


gemini-2.5-flash-lite >  Your favorite color is blue-green.


5.4 Complete Manual Workflow Test
Let's see the complete workflow in action. We'll have a conversation about a birthday, manually save it to memory, then test retrieval in a new session. This demonstrates the full cycle: ingest → store → retrieve.

In [40]:
await run_session(runner, "My birthday is on March 15th.", "birthday-session-01")


 ### Session: birthday-session-01

User > My birthday is on March 15th.
gemini-2.5-flash-lite >  Got it. I will remember that your birthday is on March 15th.


Now manually save this session to memory. This is the crucial step that transfers the conversation from short-term session storage to long-term memory storage.

In [41]:
# Manually save the session to memory
birthday_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="birthday-session-01"
)

await memory_service.add_session_to_memory(birthday_session)

Here's the crucial test: we'll start a completely new session with a different session ID and ask the agent to recall the birthday.

In [42]:
await run_session(
    runner, "When is my birthday?", "birthday-session-02"  # Different session ID
)


 ### Session: birthday-session-02

User > When is my birthday?


gemini-2.5-flash-lite >  Your birthday is on March 15th.


What happens:

Agent receives: "When is my birthday?"
Agent recognizes: This requires past conversation context
Agent calls: load_memory("birthday")
Memory returns: Previous conversation containing "March 15th"
Agent responds: "Your birthday is on March 15th"
The memory retrieval worked even though this is a completely different session!

🚀 Your Turn: Experiment with Both Patterns
Try swapping load_memory with preload_memory by changing the tools array to tools=[preload_memory].

What changes:

load_memory (reactive): Agent decides when to search
preload_memory (proactive): Automatically loads memory before every turn
Test it:

Ask "What is my favorite color?" in a new session
Ask "Tell me a joke" - notice that preload_memory still searches memory even though it's unnecessary
Which pattern is better for different use cases?
5.5 Manual Memory Search
Beyond agent tools, you can also search memories directly in your code. This is useful for:

Debugging memory contents
Building analytics dashboards
Creating custom memory management UIs
The search_memory() method takes a text query and returns a SearchMemoryResponse with matching memories.

In [43]:
# Search for color preferences
search_response = await memory_service.search_memory(
    app_name=APP_NAME, user_id=USER_ID, query="What is the user's favorite color?"
)

print("🔍 Search Results:")
print(f"  Found {len(search_response.memories)} relevant memories")
print()

for memory in search_response.memories:
    if memory.content and memory.content.parts:
        text = memory.content.parts[0].text[:80]
        print(f"  [{memory.author}]: {text}...")

🔍 Search Results:
  Found 4 relevant memories

  [user]: My favorite color is blue-green. Can you write a Haiku about it?...
  [MemoryDemoAgent]: A color so calm,
Like the ocean, cool and deep,
Nature's gentle hue....
  [user]: My birthday is on March 15th....
  [MemoryDemoAgent]: Got it. I will remember that your birthday is on March 15th....


🚀 Your Turn: Test Different Queries
Try these searches to understand how keyword matching works with InMemoryMemoryService:

"what color does the user like"
"haiku"
"age"
"preferred hue"
Notice which queries return results and which don't. What pattern do you observe?

💡 Key Insight: Memory search is grounded in reality - agents can't hallucinate memories that don't exist.

5.6 How Search Works
InMemoryMemoryService (this notebook):

Method: Keyword matching
Example: "favorite color" matches because those exact words exist
Limitation: "preferred hue" won't match
VertexAiMemoryBankService (Day 5):

Method: Semantic search via embeddings
Example: "preferred hue" WILL match "favorite color"
Advantage: Understands meaning, not just keywords
You'll explore semantic search in Day 5!

🤖 Section 6: Automating Memory Storage¶
So far, we've manually called add_session_to_memory() to transfer data to long-term storage. Production systems need this to happen automatically.

6.1 Callbacks
ADK's callback system lets you hook into key execution moments. Callbacks are Python functions you define and attach to agents - ADK automatically calls them at specific stages, acting like checkpoints during the agent's execution flow.

Think of callbacks as event listeners in your agent's lifecycle. When an agent processes a request, it goes through multiple stages: receiving the input, calling the LLM, invoking tools, and generating the response. Callbacks let you insert custom logic at each of these stages without modifying the core agent code.

Available callback types:

before_agent_callback → Runs before agent starts processing a request
after_agent_callback → Runs after agent completes its turn
before_tool_callback / after_tool_callback → Around tool invocations
before_model_callback / after_model_callback → Around LLM calls
on_model_error_callback → When errors occur
Common use cases:

Logging and observability (track what the agent does)
Automatic data persistence (like saving to memory)
Custom validation or filtering
Performance monitoring
📚 Learn More: ADK Callbacks Documentation

image.png

6.2 Automatic Memory Storage with Callbacks
For automatic memory storage, we'll use after_agent_callback. This function triggers every time the agent finishes a turn, then calls add_session_to_memory() to persist the conversation automatically.

But here's the challenge: how does our callback function actually access the memory service and current session? That's where callback_context comes in.

When you define a callback function, ADK automatically passes a special parameter called callback_context to it. The callback_context provides access to the Memory Service and other runtime components.

How we'll use it: In our callback, we'll access the memory service and current session to automatically save conversation data after each turn.

💡 Important: You don't create this context - ADK creates it and passes it to your callback automatically when the callback runs

In [44]:
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )



6.3 Create an Agent: Callback and PreLoad Memory Tool
Now create an agent that combines:

Automatic storage: after_agent_callback saves conversations
Automatic retrieval: preload_memory loads memories
This creates a fully automated memory system with zero manual intervention.

In [45]:
# Agent with automatic memory saving
auto_memory_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="AutoMemoryAgent",
    instruction="Answer user questions.",
    tools=[preload_memory],
    after_agent_callback=auto_save_to_memory,  # Saves after each turn!
)

print("✅ Agent created with automatic memory saving!")

✅ Agent created with automatic memory saving!


What happens automatically:

After every agent response → callback triggers
Session data → transferred to memory
No manual add_session_to_memory() calls needed
The framework handles everything!

6.4 Create a Runner and Test The Agent
Time to test! Create a Runner with the auto-memory agent, connecting the session and memory services.

In [46]:
# Create a runner for the auto-save agent
# This connects our automated agent to the session and memory services
auto_runner = Runner(
    agent=auto_memory_agent,  # Use the agent with callback + preload_memory
    app_name=APP_NAME,
    session_service=session_service,  # Same services from Section 3
    memory_service=memory_service,
)


In [47]:
# Test 1: Tell the agent about a gift (first conversation)
# The callback will automatically save this to memory when the turn completes
await run_session(
    auto_runner,
    "I gifted a new toy to my nephew on his 1st birthday!",
    "auto-save-test",
)

# Test 2: Ask about the gift in a NEW session (second conversation)
# The agent should retrieve the memory using preload_memory and answer correctly
await run_session(
    auto_runner,
    "What did I gift my nephew?",
    "auto-save-test-2",  # Different session ID - proves memory works across sessions!
)


 ### Session: auto-save-test

User > I gifted a new toy to my nephew on his 1st birthday!
gemini-2.5-flash-lite >  That's wonderful! A first birthday is such a special milestone. I hope your nephew loves his new toy!

 ### Session: auto-save-test-2

User > What did I gift my nephew?
gemini-2.5-flash-lite >  You gifted your nephew a new toy for his first birthday.


What just happened:

First conversation: Mentioned gift to nephew
Callback automatically saved to memory ✅
Second conversation (new session): Asked about the gift
preload_memory automatically retrieved the memory ✅
Agent answered correctly ✅
Zero manual memory calls! This is automated memory management in action.

6.5 How often should you save Sessions to Memory?¶
Options:

Timing	Implementation	Best For
After every turn	after_agent_callback	Real-time memory updates
End of conversation	Manual call when session ends	Batch processing, reduce API calls
Periodic intervals	Timer-based background job	Long-running conversations

🧩 Section 7: Memory Consolidation¶
7.1 The Limitation of Raw Storage
What we've stored so far:

Every user message
Every agent response
Every tool call
The problem:

Session: 50 messages = 10,000 tokens
Memory:  All 50 messages stored
Search:  Returns all 50 messages → Agent must process 10,000 tokens
This doesn't scale. We need consolidation.

7.2 What is Memory Consolidation?
Memory Consolidation = Extracting only important facts while discarding conversational noise.

Before (Raw Storage):

User: "My favorite color is BlueGreen. I also like purple. 
       Actually, I prefer BlueGreen most of the time."
Agent: "Great! I'll remember that."
User: "Thanks!"
Agent: "You're welcome!"

→ Stores ALL 4 messages (redundant, verbose)
After (Consolidation):

Extracted Memory: "User's favorite color: BlueGreen"

→ Stores 1 concise fact
Benefits: Less storage, faster retrieval, more accurate answers.

Memory consolidation

7.3 How Consolidation Works (Conceptual)
The pipeline:

1. Raw Session Events
   ↓
2. LLM analyzes conversation
   ↓
3. Extracts key facts
   ↓
4. Stores concise memories
   ↓
5. Merges with existing memories (deduplication)
Example transformation:

Input:  "I'm allergic to peanuts. I can't eat anything with nuts."

Output: Memory {
  allergy: "peanuts, tree nuts"
  severity: "avoid completely"
}
Natural language → Structured, actionable data.

7.4 Next Steps for Memory Consolidation
💡 Key Point: Managed Memory Services handle consolidation automatically.

You use the same API:

add_session_to_memory() ← Same method
search_memory() ← Same method
The difference: What happens behind the scenes.

InMemoryMemoryService: Stores raw events
VertexAiMemoryBankService: Intelligently consolidates before storing
📚 Learn More:

Vertex AI Memory Bank: Memory Consolidation Guide -> You'll explore this in Day 5!


📊 Summary¶
You've learned the core mechanics of Memory in ADK:

✅ Adding Memory

Initialize MemoryService alongside SessionService
Both services are provided to the Runner
✅ Storing Information

await memory_service.add_session_to_memory(session)
Transfers session data to long-term storage
Can be automated with callbacks
✅ Searching Memory

await memory_service.search_memory(app_name, user_id, query)
Returns relevant memories from past conversations
✅ Retrieving in Agents

Reactive: load_memory tool (agent decides when to use memory)
Proactive: preload_memory tool (always loads memory into LLM's system instructions)
✅ Memory Consolidation

Extracts key information from Session data
Provided by managed memory services such as Vertex AI Memory Bank